# Soporte 4 Modelo de regresión

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras

#Imrpotar datos
data = pd.read_csv('bank-full-modelo.csv')
data.head()

,default,housing,loan,pdays,job,marital,education,contact,poutcome,age,balance,day,month,duration,campaign,previous,y
0,0,1,0,0,management,married,tertiary,unknown,unknown,58,2143,5,5,261,1,0,0
1,0,1,0,0,technician,single,secondary,unknown,unknown,44,29,5,5,151,1,0,0
2,0,1,1,0,entrepreneur,married,secondary,unknown,unknown,33,2,5,5,76,1,0,0
3,0,1,0,0,blue-collar,married,unknown,unknown,unknown,47,1506,5,5,92,1,0,0
4,0,0,0,0,unknown,single,unknown,unknown,unknown,33,1,5,5,198,1,0,0


In [2]:
data.describe()

,default,housing,loan,pdays,age,balance,day,month,duration,campaign,previous,y
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,0.018027,0.555838,0.160226,0.182633,40.936210,1362.272058,15.806419,6.144655,258.163080,2.763841,0.580323,0.116985
std,0.133049,0.496878,0.366820,0.386369,10.618762,3044.765829,8.322476,2.408034,257.527812,3.098021,2.303441,0.321406
min,0.000000,0.000000,0.000000,0.000000,18.000000,-8019.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,33.000000,72.000000,8.000000,5.000000,103.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,39.000000,448.000000,16.000000,6.000000,180.000000,2.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,0.000000,48.000000,1428.000000,21.000000,8.000000,319.000000,3.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,95.000000,102127.000000,31.000000,12.000000,4918.000000,63.000000,275.000000,1.000000


In [3]:
cat_int_feats = ['default', 'housing', 'loan', 'pdays']
cat_str_feats = ['job','marital','education','contact','poutcome']
num_feats = ['age','balance', 'day', 'month', 'duration', 'campaign', 'previous']

## Separamos los datos en entrenamiento, validación y prueba

In [4]:
train = data.sample(frac=0.8, random_state=100)
train.head()

,default,housing,loan,pdays,job,marital,education,contact,poutcome,age,balance,day,month,duration,campaign,previous,y
14789,1,0,1,0,blue-collar,single,secondary,cellular,unknown,45,0,16,7,154,2,0,0
8968,0,0,0,0,management,married,tertiary,unknown,unknown,41,5,5,6,178,1,0,0
34685,0,1,0,0,management,single,secondary,cellular,unknown,40,906,5,5,67,4,0,0
2369,0,1,0,0,admin.,single,secondary,unknown,unknown,25,768,13,5,203,1,0,0
36561,0,1,0,1,services,married,primary,cellular,success,37,0,12,5,631,1,1,0


In [5]:
train.shape

(36169, 17)

In [6]:
test = data.drop(train.index)
test.head()

,default,housing,loan,pdays,job,marital,education,contact,poutcome,age,balance,day,month,duration,campaign,previous,y
9,0,1,0,0,technician,single,secondary,unknown,unknown,43,593,5,5,55,1,0,0
27,0,1,1,0,entrepreneur,married,secondary,unknown,unknown,52,113,5,5,127,1,0,0
28,0,1,0,0,management,single,secondary,unknown,unknown,46,-246,5,5,255,2,0,0
40,0,1,0,0,services,divorced,secondary,unknown,unknown,44,2586,5,5,160,1,0,0
44,0,1,0,0,retired,married,unknown,unknown,unknown,58,96,5,5,616,1,0,0


In [7]:
val = train.sample(frac=0.2, random_state=100)

In [8]:
val.shape

(7234, 17)

In [9]:
train = train.drop(val.index)

In [10]:
print(train.shape)
print(val.shape)
print(test.shape)

(28935, 17)
(7234, 17)
(9042, 17)


## Convertir de dataframe (pandas) a dataset (tensorflow)

In [11]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("y")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

In [12]:
train_ds = dataframe_to_dataset(train)
val_ds = dataframe_to_dataset(val)
test_ds = dataframe_to_dataset(test)

In [13]:
type(train_ds)

tensorflow.python.data.ops.shuffle_op._ShuffleDataset

## Separar los datos de entrenamiento, validación y prueba en lotes

In [14]:
batch_size = 32
train_ds = train_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)

## Función para codificar variables numéricas (Keras docs)

In [15]:
def encode_numerical_feature(feature, name, dataset):
    # Crea capa de normalización para este feature
    normalizer = keras.layers.Normalization()

    # Prepara el dataset para considerar únicamente la feature de interés (name)
    feature_ds = dataset.map(lambda x, y: x[name]) # selecciona variable
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1)) # deja el tensor de una dimensión

    # Aprende las estadísticas de los datos (media, varianza)
    normalizer.adapt(feature_ds)

    # Aplica la normalización a la variable
    encoded_feature = normalizer(feature)
    return encoded_feature

# Función para codificar variables categóricas (Keras docs)

In [16]:
def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = keras.layers.StringLookup if is_string else keras.layers.IntegerLookup
    # Crea una capa Lookup para retornas variables 0/1 (dummies)
    # lookup: busca el valor correspondiente de la variable categórica
    lookup = lookup_class(output_mode="binary")

    # Prepara el dataset para considerar únicamente la feature de interés (name)
    feature_ds = dataset.map(lambda x, y: x[name]) # selecciona variable
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1)) # deja el tensor de una dimensión

    # Aprende el conjunto de posibles valores que toma la variable categórica y asigna enteros
    lookup.adapt(feature_ds)

    # Aplica la conversión de categorías a enteros
    encoded_feature = lookup(feature)
    return encoded_feature

# Crear una lista de inputs para el modelo, de acuerdo con cada tipo de variable

In [17]:
inputs = []
for i in cat_int_feats:
  inputs.append(keras.Input(shape=(1,), name=i, dtype="int64"))

In [18]:
for i in cat_str_feats:
  inputs.append(keras.Input(shape=(1,), name=i, dtype="string"))

In [19]:
for i in num_feats:
  inputs.append(keras.Input(shape=(1,), name=i))

## Crear una lista de variables codificadas/normalizadas de acuerdo con su tipo, empleando las funciones de codificación/normalización

In [20]:
feats_encoded=[]

In [21]:
for i,feat in enumerate(cat_int_feats):
  feats_encoded.append(
      encode_categorical_feature(inputs[i], feat, train_ds, False)
  )

In [22]:
len_feats = len(feats_encoded)
len_feats

4

In [23]:
for i,feat in enumerate(cat_str_feats):
  feats_encoded.append(
      encode_categorical_feature(inputs[len_feats+i], feat, train_ds, True)
  )

In [24]:
len_feats = len(feats_encoded)
len_feats

9

In [25]:
for i,feat in enumerate(num_feats):
  feats_encoded.append(
      encode_numerical_feature(inputs[len_feats+i], feat, train_ds)
  )

In [26]:
for i in feats_encoded:
  print(i)

KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), name='integer_lookup/bincount/DenseBincount:0', description="created by layer 'integer_lookup'")
KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), name='integer_lookup_1/bincount/DenseBincount:0', description="created by layer 'integer_lookup_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), name='integer_lookup_2/bincount/DenseBincount:0', description="created by layer 'integer_lookup_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), name='integer_lookup_3/bincount/DenseBincount:0', description="created by layer 'integer_lookup_3'")
KerasTensor(type_spec=TensorSpec(shape=(None, 13), dtype=tf.float32, name=None), name='string_lookup/bincount/DenseBincount:0', description="created by layer 'string_lookup'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name=None), name='string_lookup_1/bi

## Creamos una capa concatenando todas las variables codificadas

In [27]:
all_feats = keras.layers.concatenate(feats_encoded)

In [28]:
type(all_feats)

keras.src.engine.keras_tensor.KerasTensor

# MODELO 1

## Capa densa con 32 neuronas y función de activación relu

In [84]:
model_layers = keras.layers.Dense(32, activation='relu')(all_feats)

## Capa de salida con 1 neurona y función de activación sigmoide

In [85]:
model_layers = keras.layers.Dense(1, activation='sigmoid')(model_layers)

## Crear el modelo con las capas ya creadas y las variables de entrada

In [86]:
model = keras.Model(inputs, model_layers)

## Compilar el modelo, definiendo optimizador, función de pérdida y métricas adicionales a capturar

In [87]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Aseguramos que Keras use TensorFlow como backend, para asegurar que el modelo pueda usar strings como entradas

In [88]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

Entrenamos el modelo con los datos en el formato tf.Dataset

In [89]:
model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50


905/905 [==============================] - 4s 3ms/step - loss: 0.2709 - accuracy: 0.8905 - val_loss: 0.2360 - val_accuracy: 0.9013
Epoch 2/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2363 - accuracy: 0.9005 - val_loss: 0.2311 - val_accuracy: 0.9043
Epoch 3/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2322 - accuracy: 0.9001 - val_loss: 0.2292 - val_accuracy: 0.9023
Epoch 4/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2293 - accuracy: 0.9022 - val_loss: 0.2264 - val_accuracy: 0.9036
Epoch 5/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2275 - accuracy: 0.9018 - val_loss: 0.2255 - val_accuracy: 0.9031
Epoch 6/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2256 - accuracy: 0.9024 - val_loss: 0.2277 - val_accuracy: 0.9035
Epoch 7/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2246 - accuracy: 0.9034 - val_loss: 0.2234 - val_accuracy: 0.9034
Epoch 8/50
905/90

## Evaluar el modelo 1

In [90]:
test_loss, test_accuracy = model.evaluate(test_ds)

print(f"Perdida: {test_loss}")
print(f"Precisión: {test_accuracy * 100:.2f}%")


283/283 [==============================] - 1s 3ms/step - loss: 0.2286 - accuracy: 0.9025
Perdida: 0.22862190008163452
Precisión: 90.25%


# MODELO 2

## 2 Capas densas con 32 neuronas y función de activación relu

In [55]:
model_layers2 = keras.layers.Dense(32, activation='relu')(all_feats)
model_layers2 = keras.layers.Dense(32, activation='relu')(all_feats)

## Capa de salida con 1 neurona y función de activación sigmoide

In [50]:
model_layers2 = keras.layers.Dense(1, activation='sigmoid')(model_layers2)

## Crear el modelo con las capas ya creadas y las variables de entrada

In [51]:
model2 = keras.Model(inputs, model_layers2)

## Compilar el modelo, definiendo optimizador, función de pérdida y métricas adicionales a capturar

In [52]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [53]:
model2.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50


905/905 [==============================] - 5s 3ms/step - loss: 0.6746 - accuracy: 0.5762 - val_loss: 0.5040 - val_accuracy: 0.8880
Epoch 2/50
905/905 [==============================] - 3s 3ms/step - loss: 0.4412 - accuracy: 0.8820 - val_loss: 0.3929 - val_accuracy: 0.8880
Epoch 3/50
905/905 [==============================] - 3s 4ms/step - loss: 0.3827 - accuracy: 0.8820 - val_loss: 0.3615 - val_accuracy: 0.8880
Epoch 4/50
905/905 [==============================] - 3s 3ms/step - loss: 0.3609 - accuracy: 0.8820 - val_loss: 0.3416 - val_accuracy: 0.8880
Epoch 5/50
905/905 [==============================] - 3s 3ms/step - loss: 0.3363 - accuracy: 0.8820 - val_loss: 0.3142 - val_accuracy: 0.8880
Epoch 6/50
905/905 [==============================] - 3s 3ms/step - loss: 0.3047 - accuracy: 0.8820 - val_loss: 0.2875 - val_accuracy: 0.8880
Epoch 7/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2770 - accuracy: 0.8820 - val_loss: 0.2671 - val_accuracy: 0.8880
Epoch 8/50
905/90

## Evaluar el modelo 2

In [54]:
test_loss, test_accuracy = model2.evaluate(test_ds)

print(f"Perdida: {test_loss}")
print(f"Precisión: {test_accuracy * 100:.2f}%")


283/283 [==============================] - 1s 2ms/step - loss: 0.2413 - accuracy: 0.8933
Perdida: 0.2413368970155716
Precisión: 89.33%


# MODELO 3

## Capa densa con 64 neuronas y función de activación tanh

In [56]:
model_layers3 = keras.layers.Dense(64, activation='tanh')(all_feats)

## Capa de salida con 1 neurona y función de activación softmax

In [57]:
model_layers3 = keras.layers.Dense(1, activation='softmax')(model_layers3)

## Crear el modelo con las capas ya creadas y las variables de entrada

In [58]:
model3 = keras.Model(inputs, model_layers3)

## Compilar el modelo, definiendo optimizador, función de pérdida y métricas adicionales a capturar

In [59]:
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [60]:
model3.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50


905/905 [==============================] - 5s 3ms/step - loss: 0.2574 - accuracy: 0.1180 - val_loss: 0.2388 - val_accuracy: 0.1120
Epoch 2/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2386 - accuracy: 0.1180 - val_loss: 0.2328 - val_accuracy: 0.1120
Epoch 3/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2345 - accuracy: 0.1180 - val_loss: 0.2312 - val_accuracy: 0.1120
Epoch 4/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2315 - accuracy: 0.1180 - val_loss: 0.2287 - val_accuracy: 0.1120
Epoch 5/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2301 - accuracy: 0.1180 - val_loss: 0.2259 - val_accuracy: 0.1120
Epoch 6/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2279 - accuracy: 0.1180 - val_loss: 0.2253 - val_accuracy: 0.1120
Epoch 7/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2259 - accuracy: 0.1180 - val_loss: 0.2287 - val_accuracy: 0.1120
Epoch 8/50
905/90

## Evaluar el modelo 3

In [61]:
test_loss, test_accuracy = model3.evaluate(test_ds)

print(f"Perdida: {test_loss}")
print(f"Precisión: {test_accuracy * 100:.2f}%")


283/283 [==============================] - 1s 2ms/step - loss: 0.2252 - accuracy: 0.1179
Perdida: 0.2251921147108078
Precisión: 11.79%


# MODELO 4

## Capa densa con 64 neuronas y función de activación tanh

In [62]:
model_layers4 = keras.layers.Dense(64, activation='tanh')(all_feats)

## Capa de salida con 1 neurona y función de activación softmax

In [63]:
model_layers4 = keras.layers.Dense(1, activation='sigmoid')(model_layers4)

## Crear el modelo con las capas ya creadas y las variables de entrada

In [64]:
model4 = keras.Model(inputs, model_layers4)

## Compilar el modelo, definiendo optimizador, función de pérdida y métricas adicionales a capturar

In [65]:
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [66]:
model4.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50


905/905 [==============================] - 4s 3ms/step - loss: 0.2606 - accuracy: 0.8917 - val_loss: 0.2365 - val_accuracy: 0.9023
Epoch 2/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2382 - accuracy: 0.8995 - val_loss: 0.2348 - val_accuracy: 0.9002
Epoch 3/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2341 - accuracy: 0.8997 - val_loss: 0.2307 - val_accuracy: 0.9028
Epoch 4/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2314 - accuracy: 0.9001 - val_loss: 0.2316 - val_accuracy: 0.9021
Epoch 5/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2297 - accuracy: 0.9013 - val_loss: 0.2258 - val_accuracy: 0.9028
Epoch 6/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2280 - accuracy: 0.9018 - val_loss: 0.2252 - val_accuracy: 0.9050
Epoch 7/50
905/905 [==============================] - 3s 4ms/step - loss: 0.2259 - accuracy: 0.9021 - val_loss: 0.2246 - val_accuracy: 0.9036
Epoch 8/50
905/90

## Evaluar el modelo 4

In [67]:
test_loss, test_accuracy = model4.evaluate(test_ds)

print(f"Perdida: {test_loss}")
print(f"Precisión: {test_accuracy * 100:.2f}%")


283/283 [==============================] - 1s 2ms/step - loss: 0.2260 - accuracy: 0.8995
Perdida: 0.2259635031223297
Precisión: 89.95%


# MODELO 5

## Capa densa con 64 neuronas y función de activación tanh

In [68]:
model_layers5 = keras.layers.Dense(64, activation='relu')(all_feats)

## Capa de salida con 1 neurona y función de activación softmax

In [69]:
model_layers5 = keras.layers.Dense(1, activation='sigmoid')(model_layers5)

## Crear el modelo con las capas ya creadas y las variables de entrada

In [70]:
model5 = keras.Model(inputs, model_layers5)

## Compilar el modelo, definiendo optimizador, función de pérdida y métricas adicionales a capturar

In [71]:
model5.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [72]:
model5.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50


905/905 [==============================] - 4s 3ms/step - loss: 0.2576 - accuracy: 0.8951 - val_loss: 0.2327 - val_accuracy: 0.9020
Epoch 2/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2328 - accuracy: 0.8989 - val_loss: 0.2280 - val_accuracy: 0.9020
Epoch 3/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2292 - accuracy: 0.9007 - val_loss: 0.2265 - val_accuracy: 0.9021
Epoch 4/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2268 - accuracy: 0.9019 - val_loss: 0.2251 - val_accuracy: 0.9032
Epoch 5/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2244 - accuracy: 0.9023 - val_loss: 0.2243 - val_accuracy: 0.9035
Epoch 6/50
905/905 [==============================] - 3s 3ms/step - loss: 0.2229 - accuracy: 0.9037 - val_loss: 0.2227 - val_accuracy: 0.9034
Epoch 7/50
905/905 [==============================] - 2s 3ms/step - loss: 0.2203 - accuracy: 0.9042 - val_loss: 0.2228 - val_accuracy: 0.9010
Epoch 8/50
905/90

## Evaluar el modelo 5

In [73]:
test_loss, test_accuracy = model5.evaluate(test_ds)

print(f"Perdida: {test_loss}")
print(f"Precisión: {test_accuracy * 100:.2f}%")


283/283 [==============================] - 1s 2ms/step - loss: 0.2293 - accuracy: 0.8977
Perdida: 0.2292981892824173
Precisión: 89.77%


## Se guarda el modelo 1 que fue el que mayor precisión tuvo

In [92]:
model.save("modelo.keras")